## Load libraries

In [1]:
import os
import requests
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt # Pendiente instalar

import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
import pprint

2025-02-26 00:14:19.724155: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 00:14:19.855604: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 00:14:20.419001: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-26 00:14:20.419086: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-26 00:14:20.524802: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## Load Data

In [2]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
 #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
 url = 'https://docs.google.com/uc?export= \
 download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
 r = requests.get(url, allow_redirects=True, stream=True)
 open(_data_filepath, 'wb').write(r.content)

# Read the CSV file into a DataFrame
if os.path.exists(_data_filepath):
    df = pd.read_csv(_data_filepath)
    print("Dataframe was loaded")  # Display the first few rows
else:
    print(f"File not found: {_data_filepath}")

Dataframe was loaded


## Feature Selection

### Data Exploration

In [4]:
# Se identifica el tipo de dato de cada una de las variables
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

In [5]:
# Se generan dos listas segun las caracteristicas de las variables para facilitar la exploracion
quantitative_variables = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points']

categorical_variables = ['Wilderness_Area', 'Soil_Type',
       'Cover_Type']

#### Quantitative Variables

In [6]:
# Se exploran las variables cuantitativas mediantes metricas de estadistica basica
df[quantitative_variables].describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
count,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000
mean,2958.442553,155.611869,14.126572,268.815297,46.540890,2344.471993,212.080110,223.328537,142.583117,1974.181002
std,280.264645,111.826494,7.517868,212.401029,58.650981,1559.288208,26.956645,19.796919,38.365816,1316.012073
min,1860.000000,0.000000,0.000000,0.000000,-166.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2808.000000,59.000000,9.000000,108.000000,7.000000,1101.000000,198.000000,213.000000,119.000000,1024.000000
50%,2996.000000,127.000000,13.000000,218.000000,29.000000,1986.000000,218.000000,226.000000,143.000000,1705.000000
75%,3163.000000,261.000000,18.000000,384.000000,69.000000,3311.000000,231.000000,237.000000,168.000000,2543.000000
max,3858.000000,360.000000,66.000000,1397.000000,598.000000,7116.000000,254.000000,254.000000,253.000000,7168.000000


In [10]:
corr_matrix = df[quantitative_variables].corr()
# Plot the heatmap
#plt.figure(figsize=(12, 8))
#sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
#plt.title("Correlation Matrix of Quantitative Variables")
#plt.show()

"""
No se identifican correlaciones directas o inversas mayores a 0.8, por consiguiente, no se descartan variables cuantitativas por correlacion
altamente significativa
"""
corr_matrix

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
Elevation,1.000000,0.013638,-0.246626,0.306427,0.089746,0.368821,0.118341,0.209270,0.056217,0.153658
Aspect,0.013638,1.000000,0.080560,0.017550,0.070517,0.024409,-0.576595,0.335843,0.646172,-0.111924
Slope,-0.246626,0.080560,1.000000,-0.011202,0.276691,-0.218410,-0.333211,-0.527148,-0.170408,-0.185317
Horizontal_Distance_To_Hydrology,0.306427,0.017550,-0.011202,1.000000,0.610122,0.074821,-0.026795,0.046380,0.052295,0.051770
Vertical_Distance_To_Hydrology,0.089746,0.070517,0.276691,0.610122,1.000000,-0.044835,-0.170377,-0.111661,0.038072,-0.070214
Horizontal_Distance_To_Roadways,0.368821,0.024409,-0.218410,0.074821,-0.044835,1.000000,0.037724,0.190562,0.104052,0.331464
Hillshade_9am,0.118341,-0.576595,-0.333211,-0.026795,-0.170377,0.037724,1.000000,0.013680,-0.780337,0.135143
Hillshade_Noon,0.209270,0.335843,-0.527148,0.046380,-0.111661,0.190562,0.013680,1.000000,0.590911,0.056642
Hillshade_3pm,0.056217,0.646172,-0.170408,0.052295,0.038072,0.104052,-0.780337,0.590911,1.000000,-0.050605
Horizontal_Distance_To_Fire_Points,0.153658,-0.111924,-0.185317,0.051770,-0.070214,0.331464,0.135143,0.056642,-0.050605,1.000000


In [11]:
for i in categorical_variables:
    print(f"variable: {i}")
    print(df[i].value_counts())
    print("--------------------")
"""
Las variables categoricas estan altamente desbalaceadas, en especial "Wilderness_Area" y "Cover_Type" se concentran principalmente en 2 variables.
Por su parte, "Soil_Type" se concentra principalmente en 4 variables, por la cantidad de categorias, hay algunas con pocos valores.
"""
    

variable: Wilderness_Area
Rawah        52006
Commanche    50759
Cache         7525
Neota         5913
Name: Wilderness_Area, dtype: int64
--------------------
variable: Soil_Type
C7745    22912
C7202    11560
C7756    10539
C7757     9178
C7201     6690
C4703     6541
C7746     6068
C4744     5961
C7755     5130
C7700     4244
C4758     3443
C8771     3113
C8772     2720
C2705     2488
C4704     2396
C7102     1881
C8776     1729
C2703     1478
C2717     1378
C2704      977
C7101      787
C6102      690
C2702      592
C6101      565
C7702      539
C8703      385
C6731      375
C7790      324
C2706      322
C4201      257
C7709      242
C7710      194
C7103      156
C5101      123
C7701      101
C8708       63
C3502       24
C3501       19
C8707       18
C5151        1
Name: Soil_Type, dtype: int64
--------------------
variable: Cover_Type
1    56720
0    42307
2     7228
6     4045
5     3478
4     1892
3      533
Name: Cover_Type, dtype: int64
--------------------


'\nLas variables categoricas estan altamente desbalaceadas, en especial "Wilderness_Area" y "Cover_Type" se concentran principalmente en 2 variables.\nPor su parte, "Soil_Type" se concentra principalmente en 4 variables, por la cantidad de categorias, hay algunas con pocos valores.\n'

In [12]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(df[categorical_variables])

ModuleNotFoundError: No module named 'sklearn'